# Erik Holmgren
## 2018-09-18
## CS 425: Assignment 1
## Dr. Hamid Chitsaz

In [2]:
import numpy as np

### Global alignment with limited gaps

In [19]:
def global_alignment(u, v, delta, k=-1):
    u = u.upper()
    v = v.upper()
    n = len(u)
    m = len(v)
    f = np.zeros((n + 1, m + 1))
    d = np.zeros((n + 1, m + 1), dtype=str)


    def scoref(i, j):
        vals = []
        vals.append((f[i - 1, j - 1] + delta[u[i - 1]][v[j - 1]], 'd'))
        vals.append((f[i - 1, j] + delta[u[i - 1]]['-'], 'l'))
        vals.append((f[i, j - 1] + delta['-'][v[j - 1]], 'u'))

        return  max(vals, key=lambda x: x[0])


    for i in range(1, n + 1):
        f[i, 0] = -1 * i

    for j in range(1, m + 1):
        f[0, j] = -1 * j

    d[0, 1:] = 'l'
    d[1:, 0] = 'u'

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            f[i, j], d[i, j] = scoref(i, j)

    u_aligned = []
    v_aligned = []
    
    if k == -1:
        k = float('inf')

    u_gap_count = k
    v_gap_count = k

    score = 0
    i = n
    j = m
    dirr = d[i, j]
    while i > 0 or j > 0:
        if dirr == 'l' and u_gap_count > 0:
            u_aligned.insert(0, '-')
            v_aligned.insert(0, v[j - 1])
            j -= 1
            u_gap_count -= 1
        elif dirr == 'u' and v_gap_count > 0:
            u_aligned.insert(0, u[i - 1])
            v_aligned.insert(0, '-')
            i -= 1
            v_gap_count -= 1
        else: # dirr == 'd':
            u_aligned.insert(0, u[i - 1])
            v_aligned.insert(0, v[j - 1])
            i -= 1
            j -= 1
            u_gap_count = k
            v_gap_count = k

        score += f[i, j]
        dirr = d[i, j]

    return ''.join(u_aligned), ''.join(v_aligned), score

In [20]:
global_match = 1
global_sigma = -1
global_mu = -2

global_score_matrix = {
    'A' : {'A' : global_match, 'T': global_mu, 'C' : global_mu, 'G' : global_mu, '-': global_sigma},
    'T' : {'A' : global_mu, 'T': global_match, 'C' : global_mu, 'G' : global_mu, '-': global_sigma},
    'C' : {'A' : global_mu, 'T': global_mu, 'C' : global_match, 'G' : global_mu, '-': global_sigma},
    'G' : {'A' : global_mu, 'T': global_mu, 'C' : global_mu, 'G' : global_match, '-': global_sigma},
    '-' : {'A' : global_sigma, 'T': global_sigma, 'C' : global_sigma, 'G' : global_sigma, '-': None},
}

In [21]:
import random

def randomDNA(length):
    nucs = ['A', 'C', 'T', 'G']
    return ''.join([random.choice(nucs) for i in range(length)])

In [24]:
u = randomDNA(24)
v = randomDNA(24)

w, x, score = global_alignment(u, v, global_score_matrix, k=3)
print(f'{w}\n{x}\nscore: {score}\n')

w, x, score = global_alignment(u, v, global_score_matrix, k=2)
print(f'{w}\n{x}\nscore: {score}\n')

w, x, score = global_alignment(u, v, global_score_matrix, k=1)
print(f'{w}\n{x}\nscore: {score}\n')

w, x, score = global_alignment(u, v, global_score_matrix, k=0)
print(f'{w}\n{x}\nscore: {score}\n')

w, x, score = global_alignment(u, v, global_score_matrix, k=-1)
print(f'{w}\n{x}\nscore: {score}\n')

ATTAACTT-T-CC-CGTGG-GGTAAGG--T-
---G---GCAACATCGTCCTGCCCCGGAAAG
score: -270.0

ATTAACTT-T-CC-CGTGG-GGTAAGG--T-
AA--G--GCAACATCGTCCTGCCCCGGAAAG
score: -279.0

ATTAACTT-T-CC-CGTGG-GGTAA-GG-T-
AA-A-G-GCAACATCGTCCTGCCCCGGAAAG
score: -282.0

ATTAACTTTCCCGTGGGGTAAGGT
GCAACATCGTCCTGCCCCGGAAAG
score: -108.0

ATTAACTT-T-CC-CGTGG-GGTAAGG--T-
-------GCAACATCGTCCTGCCCCGGAAAG
score: -192.0



### Semi-global Alignment

In [7]:
def semi_global_alignment(u, v, delta):
    u = u.upper()
    v = v.upper()

    n, m = len(u), len(v)

    f = np.zeros((n + 1, m + 1))
    d = np.zeros((n + 1, m + 1), dtype=str)

    def score(i, j):
        vals = []
        vals.append((f[i - 1, j - 1] + delta[u[i - 1]][v[j - 1]], 'd'))
        vals.append((f[i - 1, j] + delta[u[i - 1]]['-'], 'l'))
        vals.append((f[i, j - 1] + delta['-'][v[j - 1]], 'u'))

        return max(vals, key=lambda x: x[0])


    for i in range(1, n + 1):
        f[i, 0] = -1 * i

    d[0, 1:] = 'l'
    d[1:,0] = 'u'

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            f[i, j], d[i, j] = score(i, j)


    u_aligned = []
    v_aligned = []

    score = 0
    i = n
    j = m

    max_row = (f[0, m], 0)
    for c in range(n):
        if f[c, m] > max_row[0]:
            max_row = f[c, m], c

    i = max_row[1]

    for c in range(n - 1, i - 1, -1):
        u_aligned.insert(0, u[c])
        v_aligned.insert(0, '-')

    dirr = d[i, j]
    while i > 0 or j > 0:
        if dirr == 'd':
            u_aligned.insert(0, u[i - 1])
            v_aligned.insert(0, v[j - 1])
            i -= 1
            j -= 1
        elif dirr == 'l':
            u_aligned.insert(0, '-')
            v_aligned.insert(0, v[j - 1])
            j -= 1
        elif dirr == 'u':
            u_aligned.insert(0, u[i - 1])
            v_aligned.insert(0, '-')
            i -= 1

        score += f[i, j]
        dirr = d[i, j]

    return ''.join(u_aligned), ''.join(v_aligned), score

In [8]:
u = 'CAGCACTTGGATTCTCGG'
v = 'CAGCGTGG'

u, v, score = semi_global_alignment(u, v, global_score_matrix)

print(f'{u}\n{v}\nscore: {score}')

CAGCA-CTTGGATTCTCGG
---CAGCGTGG--------
score: -3.0


## Local Alignment

In [9]:
def local_alignment(u, v, delta):
    u = u.upper()
    v = v.upper()
    
    n, m = len(u), len(v)
    f = np.zeros((n + 1, m + 1))
    d = np.zeros((n + 1, m + 1), dtype=str)
    
    def score(i, j):
        vals = []
        #vals.append((f[i - 1, j - 1] + delta[u[i - 1]][v[j - 1]], 'd'))
        #vals.append((max([f[i - k, j] + (k * delta[u[i - 1]]['-']) for k in range(1, i + 1)]), 'u'))
        #vals.append((max([f[i, j - k] + (k * delta['-'][v[j - 1]]) for k in range(1, j + 1)]), 'l'))
        
        vals.append((f[i - 1, j - 1] + delta[u[i - 1]][v[j - 1]], 'd'))
        vals.append((f[i, j - 1] + delta['-'][v[j - 1]], 'l'))
        vals.append((f[i - 1, j] + delta[u[i - 1]]['-'], 'u'))
        
        best = max(vals, key= lambda x: x[0])
        
        if best[0] < 0:
            return 0, best[1]
        
        return best

    
    d[0, 1:] = 'l'
    d[1:,0] = 'u'
    
    best_yet = (0, 0, 0)
    for i in range (1, n + 1):
        for j in range(1, m + 1):
            f[i, j], d[i, j] = score(i, j)
            if f[i, j] > best_yet[0]:
                best_yet = (f[i, j], i, j)
            
    u_aligned = []
    v_aligned = []
    
    score = 0
    _, i, j = best_yet
    dirr = d[i, j]
    while i > 0 or j > 0:
        if f[i, j] == 0:
            break
        if dirr == 'd':
            u_aligned.insert(0, u[i - 1])
            v_aligned.insert(0, v[j - 1])
            i -= 1
            j -= 1
        elif dirr == 'l':
            u_aligned.insert(0, '-')
            v_aligned.insert(0, v[j - 1])
            j -= 1
        elif dirr == 'u':
            u_aligned.insert(0, u[i - 1])
            v_aligned.insert(0, '-')
            i -= 1
            
        score += f[i, j]
        dirr = d[i, j]
        
    return ''.join(u_aligned), ''.join(v_aligned), score

In [10]:
u = 'TGTTACGG'
v = 'GGTTGACTA'

u, v, score = local_alignment(u, v, global_score_matrix)

print(f'{u}\n{v}\nscore: {score}')

GTT-AC
GTTGAC
score: 11.0


In [11]:
b62 = '''
   A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *
A  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4 
R -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4 
N -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4 
D -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4 
C  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4 
Q -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4 
E -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4 
G  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4 
H -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4 
I -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3 -3 -1 -4 
L -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4 -3 -1 -4 
K -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0  1 -1 -4 
M -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3 -1 -1 -4 
F -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1 -3 -3 -1 -4 
P -1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2 -2 -1 -2 -4 
S  1 -1  1  0 -1  0  0  0 -1 -2 -2  0 -1 -2 -1  4  1 -3 -2 -2  0  0  0 -4 
T  0 -1  0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1  1  5 -2 -2  0 -1 -1  0 -4 
W -3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1  1 -4 -3 -2 11  2 -3 -4 -3 -2 -4 
Y -2 -2 -2 -3 -2 -1 -2 -3  2 -1 -1 -2 -1  3 -3 -2 -2  2  7 -1 -3 -2 -1 -4 
V  0 -3 -3 -3 -1 -2 -2 -3 -3  3  1 -2  1 -1 -2 -2  0 -3 -1  4 -3 -2 -1 -4 
B -2 -1  3  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3  4  1 -1 -4 
Z -1  0  0  1 -3  3  4 -2  0 -3 -3  1 -1 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4 
X  0 -1 -1 -1 -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -2  0  0 -2 -1 -1 -1 -1 -1 -4 
* -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  1
'''
legend = [c if c != '*' else '-' for c in b62.splitlines()[1].split()]
#legend

In [12]:
blos62 = dict()

for line in b62.splitlines()[2:]:
    spl = line.split()
    key = spl.pop(0)
    if key == '*':
        key = '-'
    blos62[key] = dict()
    for i in range(len(spl)):
        blos62[key][legend[i]] = int(spl[i]) if spl[i] != '*' else '-'
    
#blos62

In [13]:
aniridia = '''
MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY
YETGSIRPRAIGGSKPRVATPEVVSKIAQYKRECPSIFAWEIRDRLLSEGVCTNDNIPSV
SSINRVLRNLASEKQQMGADGMYDKLRMLNGQTGSWGTRPGWYPGTSVPGQPTQDGCQQQ
EGGGENTNSISSNGEDSDEAQMRLQLKRKLQRNRTSFTQEQIEALEKEFERTHYPDVFAR
ERLAAKIDLPEARIQVWFSNRRAKWRREEKLRNQRRQASNTPSHIPISSSFSTSVYQPIP
QPTTPVSSFTSGSMLGRTDTALTNTYSALPPMPSFTMANNLPMQPPVPSQTSSYSCMLPT
SPSVNGRSYDTYTPPHMQTHMNSQPMGTSGTTSTGLISPGVSVPVQVPGSEPDMSQYWPR
LQ
'''.replace('\n', '')

eyeless = '''
MRNLPCLGTAGGSGLGGIAGKPSPTMEAVEASTASHRHSTSSYFATTYYHLTDDECHSGV
NQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRP
RAIGGSKPRVATAEVVSKISQYKRECPSIFAWEIRDRLLQENVCTNDNIPSVSSINRVLR
NLAAQKEQQSTGSGSSSTSAGNSISAKVSVSIGGNVSNVASGSRGTLSSSTDLMQTATPL
NSSESGGASNSGEGSEQEAIYEKLRLLNTQHAAGPGPLEPARAAPLVGQSPNHLGTRSSH
PQLVHGNHQALQQHQQQSWPPRHYSGSWYPTSLSEIPISSAPNIASVTAYASGPSLAHSL
SPPNDIESLASIGHQRNCPVATEDIHLKKELDGHQSDETGSGEGENSNGGASNIGNTEDD
QARLILKRKLQRNRTSFTNDQIDSLEKEFERTHYPDVFARERLAGKIGLPEARIQVWFSN
RRAKWRREEKLRNQRRTPNSTGASATSSSTSATASLTDSPNSLSACSSLLSGSAGGPSVS
TINGLSSPSTLSTNVNAPTLGAGIDSSESPTPIPHIRPSCTSDNDNGRQSEDCRRVCSPC
PLGVGGHQNTHHIQSNGHAQGHALVPAISPRLNFNSGSFGAMYSNMHHTALSMSDSYGAV
TPIPSFNHSAVGPLAPPSPIPQQGDLTPSSLYPCHMTLRPPPMAPAHHHIVPGDGGRPAG
VGLGSGQSANLGASCSGSGYEVLSAYALPPPPMASSSAADSSFSAASSASANVTPHHTIA
QESCPSPCSSASHFGVAHSSGFSSDPISPAVSSYAHMSYNYASSANTMTPSSASGTSAHV
APGKQQFFASCFYSPWV
'''.replace('\n', '')

In [14]:
aniridia_aligned, eyeless_aligned, an_vs_ey_score = local_alignment(aniridia, eyeless, blos62)

In [15]:
print(f'{aniridia_aligned}\n\n{eyeless_aligned}\n\nscore: {an_vs_ey_score}')

HSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATPEVVSKIAQYKRECPSIFAWEIRDRLLSEGVCTNDNIPSVSSINRVLRNLASEKQQMGADGMYDKLRMLNGQTGSWGTRPGWYPGTSVPGQPTQDGCQQQEGGGENTNSISSNGEDSDEAQMRLQLKRKLQRNRTSFTQEQIEAL-EK-EFERT-H-Y-PDVFARERLAAKI-DLPEARIQVWFSNRRAKWRREEKLRNQRRQASNTPSHIPISSSFSTSVYQ-PIPQ-PT-TPVSSFTSGSMLGRTDTALTNTYSAL

HSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATAEVVSKISQYKRECPSIFAWEIRDRLLQENVCTNDNIPSVSSINRVLRNLAAQKEQQ-STG---S-G--SSST-SAGNSISAKVSVSIGGN-VSN-V---ASGSRGT--LSSS-TDLMQTATPLNSSESGGASNSGEGSEQ-EAIYEKLRLLNTQHAAGPGPLEPARAAPLVGQSPN-HLGTRSSHPQLVHGNHQALQ-QHQQQSWPPRHYS-GSWYPTSLSEIPISSAPNIASVTAYASGPSLAHS-LSPPNDIESL

score: 171207.0


In [16]:
def show_pairing(seq1, seq2, printlength=80):
    assert len(seq1) == len(seq2)
    line1 = str()
    line2 = str()
    line3 = str()
    for i in range(len(seq1)):
        line1 = line1 + seq1[i]
        line2 = line2 + (' ' if seq1[i] == seq2[i] else 'X')
        line3 = line3 + seq2[i]
        
        if len(line1) != 0 and len(line1) > printlength:
            print(line1)
            print(line2)
            print(line3)
            print()
            line1 = ''
            line2 = ''
            line3 = ''

In [17]:
show_pairing(aniridia_aligned, eyeless_aligned)

HSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATPEVVS
            X                                                               X    
HSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATAEVVS

KIAQYKRECPSIFAWEIRDRLLSEGVCTNDNIPSVSSINRVLRNLASEKQQMGADGMYDKLRMLNGQTGSWGTRPGWYPGT
  X                   X X                     XX X XXXX XXXXXXXXXXX X X XXXXXXXXX
KISQYKRECPSIFAWEIRDRLLQENVCTNDNIPSVSSINRVLRNLAAQKEQQ-STG---S-G--SSST-SAGNSISAKVSV

SVPGQPTQDGCQQQEGGGENTNSISSNGEDSDEAQMRLQLKRKLQRNRTSFTQEQIEAL-EK-EFERT-H-Y-PDVFARER
 XX XXXXXXXXXXXX XXX XXX  XXX XXXXXXX XXXXXXXXXXXXXXX  X  XX  XXXXX X XXX XXXXXX 
SIGGN-VSN-V---ASGSRGT--LSSS-TDLMQTATPLNSSESGGASNSGEGSEQ-EAIYEKLRLLNTQHAAGPGPLEPAR

LAAKI-DLPEARIQVWFSNRRAKWRREEKLRNQRRQASNTPSHIPISSSFSTSVYQ-PIPQ-PT-TPVSSFTSGSMLGRTD
X XXXXXX XXXXXXXX XXXXXXXXXXX XX XX X XX X XXXX XXX  XXXX  XXX XXXX XXXX  XX XXXX
AAPLVGQSPN-HLGTRSSHPQLVHGNHQALQ-QHQQQSWPPRHYS-GSWYPTSLSEIPISSAPNIASVTAYASGPSLAHS-



In [18]:
l = len(aniridia)
chars = list(set(aniridia))
random_scores = []

for i in range(100):
    u = ''.join([random.choice(chars) for i in range(l)])
    _, _, score = local_alignment(u, eyeless, blos62)
    random_scores.append(score)
    print(f'{i}: {score}, ', end='')
    if i % 6 == 0 and i != 0:
        print()


0: 5284.0, 1: 2853.0, 2: 2851.0, 3: 4881.0, 4: 2345.0, 5: 8817.0, 6: 7016.0, 
7: 4541.0, 8: 6535.0, 9: 8949.0, 10: 5822.0, 11: 13154.0, 12: 6371.0, 
13: 3985.0, 14: 7932.0, 15: 5879.0, 16: 6218.0, 17: 2017.0, 18: 10960.0, 
19: 4409.0, 20: 7524.0, 21: 1521.0, 22: 5833.0, 23: 4862.0, 24: 4844.0, 
25: 12453.0, 26: 6061.0, 27: 4649.0, 28: 12911.0, 29: 6153.0, 30: 4049.0, 
31: 6395.0, 32: 12768.0, 33: 1832.0, 34: 2471.0, 35: 3537.0, 36: 4099.0, 
37: 8442.0, 38: 6696.0, 39: 5031.0, 40: 21162.0, 41: 12566.0, 42: 6843.0, 
43: 2336.0, 44: 769.0, 45: 3055.0, 46: 6488.0, 47: 5877.0, 48: 5257.0, 
49: 6783.0, 50: 10784.0, 51: 7596.0, 52: 8769.0, 53: 7737.0, 54: 7881.0, 
55: 11013.0, 56: 3160.0, 57: 4635.0, 58: 8178.0, 59: 14234.0, 60: 15347.0, 
61: 3570.0, 62: 10793.0, 63: 7160.0, 64: 5482.0, 65: 3854.0, 66: 4691.0, 
67: 17068.0, 68: 7994.0, 69: 10832.0, 70: 3232.0, 71: 3913.0, 72: 8622.0, 
73: 25234.0, 74: 5604.0, 75: 6554.0, 76: 16802.0, 77: 1423.0, 78: 5849.0, 
79: 3797.0, 80: 8547.0, 81: 10953.

In [16]:
import statistics

statistics.mean(random_scores), statistics.stdev(random_scores)

(6814.19, 4042.8792112887454)

In [1]:
from statsmodels.stats.weightstats import ttest_ind
import statsmodels

tstat, pval, df = statsmodels.stats.weightstats.ttest_ind([an_vs_ey_score], random_scores, alternative='larger')
print(f'Our alignment score is highly signifigant at the {pval} p-value, (test statistic: {tstat}, degrees of freedom: {df})')

NameError: name 'an_vs_ey_score' is not defined